In [1]:
import optuna
import os
from pathlib import Path
import kaggle
from catboost import CatBoostClassifier
import pandas as pd
import numpy as np
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
if iskaggle:
    path = Path('../input/playground-series-s3e23')
    orig_path = Path('../input/software-defect-prediction')
else:
    path = Path('../input/playground-series-s3e23')
    orig_path = Path('../input/software-defect-prediction')
    if not path.exists():
        kaggle.api.competition_download_cli(str(path))
        zipfile.ZipFile(f'{path}.zip').extractall(path)
    if not orig_path.exists():
        kaggle.api.competition_download_cli(str(path))
        zipfile.ZipFile(f'{path}.zip').extractall(path)

SEED = 42
SPLITS = 5
VERSION = 3
OUTPUT = f'cat_boost_clf_submisson_v{VERSION}.csv'

np.random.seed(SEED)

/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train = pd.read_csv(path/'train.csv', index_col = 'id')
test = pd.read_csv(path/'test.csv', index_col = 'id')
orig_train = pd.read_csv(orig_path/'jm1.csv')

In [3]:
for object_features in list(orig_train.loc[:, orig_train.dtypes == 'O']):
    orig_train[object_features] = orig_train[object_features].replace({'?' : np.nan}).astype('float64')

In [4]:
X = pd.concat([train, orig_train])
y = X.pop('defects')

In [5]:
partial_pipeline = make_pipeline(SimpleImputer(), FunctionTransformer(func=np.log1p, validate=False), StandardScaler())

In [6]:
def objective(trial):
    params = {
        'task_type', 'GPU',
        'n_estimators': trial.suggest_int('n_estimators',50,1000, log=True),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1, log=True),
        'depth': trial.suggest_int('depth', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.05, 1.0),
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.05, 1.0),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 100),
        'verbose': trial.suggest_categorical('verbose', [False])
    }
    pipeline = make_pipeline(partial_pipeline, CatBoostClassifier(**params))
    scores = cross_val_score(pipeline, X, y, cv=SPLITS, scoring='roc_auc', n_jobs=-1)
    return scores.mean()

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30, show_progress_bar=True)

[I 2023-10-14 22:18:31,653] A new study created in memory with name: no-name-e9c16479-eeb4-4867-a977-d811a79abc0f
Best trial: 0. Best value: 0.775943:   3%|█████▍                                                                                                                                                              | 1/30 [00:29<14:05, 29.15s/it]

[I 2023-10-14 22:19:00,802] Trial 0 finished with value: 0.7759434200883465 and parameters: {'n_estimators': 260, 'learning_rate': 0.004006292778237605, 'depth': 9, 'subsample': 0.546649559330401, 'colsample_bylevel': 0.11451087804995956, 'min_data_in_leaf': 24, 'verbose': False}. Best is trial 0 with value: 0.7759434200883465.


Best trial: 1. Best value: 0.776006:   7%|██████████▉                                                                                                                                                         | 2/30 [00:36<07:32, 16.16s/it]

[I 2023-10-14 22:19:07,870] Trial 1 finished with value: 0.7760057238661118 and parameters: {'n_estimators': 51, 'learning_rate': 0.001813377714006975, 'depth': 7, 'subsample': 0.31639266729391397, 'colsample_bylevel': 0.664959651401588, 'min_data_in_leaf': 13, 'verbose': False}. Best is trial 1 with value: 0.7760057238661118.


Best trial: 1. Best value: 0.776006:  10%|████████████████▍                                                                                                                                                   | 3/30 [00:50<06:55, 15.40s/it]

...

[I 2023-10-14 22:33:37,495] Trial 27 finished with value: 0.7852990597094218 and parameters: {'n_estimators': 286, 'learning_rate': 0.07110901618444579, 'depth': 5, 'subsample': 0.4940134861055049, 'colsample_bylevel': 0.9090142712877507, 'min_data_in_leaf': 62, 'verbose': False}. Best is trial 27 with value: 0.7852990597094218.


Best trial: 27. Best value: 0.785299:  97%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌     | 29/30 [15:22<00:30, 30.82s/it]

[I 2023-10-14 22:33:54,424] Trial 28 finished with value: 0.7841275549958918 and parameters: {'n_estimators': 229, 'learning_rate': 0.031036545216701076, 'depth': 5, 'subsample': 0.47397145939865326, 'colsample_bylevel': 0.9019512672309632, 'min_data_in_leaf': 63, 'verbose': False}. Best is trial 27 with value: 0.7852990597094218.


Best trial: 27. Best value: 0.785299: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [15:45<00:00, 31.51s/it]

[I 2023-10-14 22:34:16,898] Trial 29 finished with value: 0.7848925940470541 and parameters: {'n_estimators': 301, 'learning_rate': 0.07102455993430254, 'depth': 3, 'subsample': 0.5511355510787153, 'colsample_bylevel': 0.9213696062365356, 'min_data_in_leaf': 48, 'verbose': False}. Best is trial 27 with value: 0.7852990597094218.


In [7]:
study.best_params, study.best_value

({'n_estimators': 286,
  'learning_rate': 0.07110901618444579,
  'depth': 5,
  'subsample': 0.4940134861055049,
  'colsample_bylevel': 0.9090142712877507,
  'min_data_in_leaf': 62,
  'verbose': False},
 0.7852990597094218)

In [8]:
pipeline = make_pipeline(partial_pipeline, CatBoostClassifier(**study.best_params))
pipeline.fit(X, y)

Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('simpleimputer', SimpleImputer()),
                                 ('functiontransformer',
                                  FunctionTransformer(func=<ufunc 'log1p'>)),
                                 ('standardscaler', StandardScaler())])),
                ('catboostclassifier',
                 <catboost.core.CatBoostClassifier object at 0x7f548da32c50>)])

In [9]:
submission = test.copy()
submission['defects'] = pipeline.predict_proba(submission)[:, 1]
submission.defects.to_csv(OUTPUT)